In [1]:
from typing import Dict, List, Tuple
import os
import torch
from torch.utils.data import DataLoader
import torchvision
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms
from modular_classification import (data_setup, 
                                    engine)
from modular_classification.engine import test_func                                 
import torchinfo
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
from datetime import datetime

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
def set_seed(seed: int=23):
    """
    Sets a specific seed for the tests
    
    Args:
        seed (int, optional): Random seed to set. Defaults is 23.
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [4]:
test_dir= "./datasets/pizza_sushi/test"
train_dir = "./datasets/pizza_sushi/train"
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transforms = weights.transforms()
train_dataloader, test_dataloader,class_names = data_setup.create_dataloaders(train_dir=train_dir
                                                                               , test_dir=test_dir
                                                                                ,transform=auto_transforms,
                                                                                  batch_size=32)
model = torchvision.models.efficientnet_b0(weights=weights).to(device)                                                                          
for param in model.features.parameters():
    param.requires_grad = False

torch.manual_seed(23)
torch.cuda.manual_seed(23)

output_shape = len(class_names)

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), #We dont change this variable
    torch.nn.Linear(in_features=1280, out_features=output_shape, bias=True)
).to(device)

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
train_dataloader.dataset, test_dataloader.dataset

(Dataset ImageFolder
     Number of datapoints: 450
     Root location: ./datasets/pizza_sushi/train
     StandardTransform
 Transform: ImageClassification(
                crop_size=[224]
                resize_size=[256]
                mean=[0.485, 0.456, 0.406]
                std=[0.229, 0.224, 0.225]
                interpolation=InterpolationMode.BICUBIC
            ),
 Dataset ImageFolder
     Number of datapoints: 150
     Root location: ./datasets/pizza_sushi/test
     StandardTransform
 Transform: ImageClassification(
                crop_size=[224]
                resize_size=[256]
                mean=[0.485, 0.456, 0.406]
                std=[0.229, 0.224, 0.225]
                interpolation=InterpolationMode.BICUBIC
            ))

In [7]:
torchinfo.summary(model,(32, 3, 224, 224), col_names=["num_params","trainable"]
                  ,row_settings=["var_names"])

Layer (type (var_name))                                      Param #                   Trainable
EfficientNet (EfficientNet)                                  --                        Partial
├─Sequential (features)                                      --                        False
│    └─Conv2dNormActivation (0)                              --                        False
│    │    └─Conv2d (0)                                       (864)                     False
│    │    └─BatchNorm2d (1)                                  (64)                      False
│    │    └─SiLU (2)                                         --                        --
│    └─Sequential (1)                                        --                        False
│    │    └─MBConv (0)                                       (1,448)                   False
│    └─Sequential (2)                                        --                        False
│    │    └─MBConv (0)                                       (6,004

In [11]:
writer = SummaryWriter()

We need to modify our existing train and test function to integrate the write funtion of the tensorboard writer, its usage is **writer.add_scalars(main_tag, tag_scalar_dict)** \
main_tag = the metric
tag scalar = a dict with the results like {"train_loss": int}

In [16]:
def train(model: nn.Module, test_data: DataLoader, train_data: DataLoader, loss_fn:nn.Module,
        optimizer: torch.optim.Optimizer, device: torch.device, epochs: int) -> Dict[str, List]:
    """
    Trains and test a Pytorch model

    Args:
    -----
        model: A Pytorch model
        train_data: a DataLoader object with the train data
        test_data: a DataLoader object with the train data
        loss_fn: loss function to minimized
        optimizer: A Pytorch optimizer
        device: A target device to perform the operations ("cuda" or "cpu")
        epochs: A integre with the number of epochs that the model will be train
    Returns:
    --------
        A dictionary with the train and test loss and accuracy for every epoch
        in the form of 
        {"train_loss": [...],
        "train_acc": [...],
        "test_loss": [...],
        "test_acc": [...]}
    """
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_func(
            model,train_data,loss_fn,optimizer, device)
        test_loss, test_acc = test_func(
            model, test_data, loss_fn, device)
        print(
            f"Epoch {epoch+1} |"
            f"train_loss :{train_loss: .4f} |"
            f"train_acc :{train_acc: .4f} |"
            f"test_loss :{test_loss: .4f} |"
            f"test_acc :{test_acc: .4f} "
        )
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ##Add of the writer
        writer.add_scalars(main_tag="Loss", 
        tag_scalar_dict={"train_loss": train_loss, "tests_loss": test_loss},
        global_step=epoch
        )

        writer.add_scalars(main_tag="Accuracy", 
        tag_scalar_dict={"train_acc": train_acc, "tests_acc": test_acc},
        global_step=epoch
        )

        writer.close()
    return results

In [21]:
set_seed()
results = train(model, test_dataloader, train_dataloader, loss_fn, optimizer, device, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 |train_loss : 1.3345 |train_acc : 0.3063 |test_loss : 0.7612 |test_acc : 0.6557 
Epoch 2 |train_loss : 0.4806 |train_acc : 0.9333 |test_loss : 0.3874 |test_acc : 0.9534 
Epoch 3 |train_loss : 0.3952 |train_acc : 0.9187 |test_loss : 0.2825 |test_acc : 0.9597 
Epoch 4 |train_loss : 0.3089 |train_acc : 0.9375 |test_loss : 0.2341 |test_acc : 0.9722 
Epoch 5 |train_loss : 0.2478 |train_acc : 0.9521 |test_loss : 0.2055 |test_acc : 0.9722 
Epoch 6 |train_loss : 0.2096 |train_acc : 0.9583 |test_loss : 0.1836 |test_acc : 0.9722 
Epoch 7 |train_loss : 0.1834 |train_acc : 0.9646 |test_loss : 0.1668 |test_acc : 0.9722 
Epoch 8 |train_loss : 0.1632 |train_acc : 0.9708 |test_loss : 0.1538 |test_acc : 0.9722 
Epoch 9 |train_loss : 0.1467 |train_acc : 0.9729 |test_loss : 0.1435 |test_acc : 0.9722 
Epoch 10 |train_loss : 0.1328 |train_acc : 0.9812 |test_loss : 0.1351 |test_acc : 0.9722 


Lets defying a function that creates a writer base in time and other parameters

In [6]:
def create_write(name: str, model: str, extra: str=None) -> SummaryWriter():
    from datetime import datetime
    import os

    timestamp = datetime.now().strftime("%Y-%m-%d")
    if extra:
        log_dir = os.path.join("runs", timestamp, name, model, extra)
    else:
        log_dir = os.path.join("runs", timestamp, name, model)
    print(f"[INFO] create summary writer, saving to: {log_dir}")
    return SummaryWriter(log_dir=log_dir)

Updating the train function

In [15]:
def train_func(model: nn.Module, data: DataLoader, loss_fn:nn.Module,
               optimizer: torch.optim.Optimizer, device: torch.device
               , log_dir: str) -> Tuple[float, float]:
    """
    Trains a model for a single epoch

    Args
    ----
        model: A Pytorch model
        data: a DataLoader object with the train data
        loss_fn: loss function to minimized
        optimizer: A Pytorch optimizer
        device: A target device to perform the operations ("cuda" or "cpu")

    Returns
    ------
        A tuple with the loss and accuracy of the training epoch like
        (train_loss, train_acc)
    """
    def trace_handler(prof):
        print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=-1))

    model.train()
    train_loss, train_acc = 0, 0
    with torch.profiler.profile(
        schedule=torch.profiler.schedule(
            wait=1,
            warmup=1,
            active=1,
            repeat=1),
        #on_trace_ready=trace_handler,
        on_trace_ready=torch.profiler.tensorboard_trace_handler("./log"),
        with_stack=True
    ) as profiler:
        for _ , (x, y) in enumerate(data):
            x, y = x.to(device), y.to(device)
            y_logit = model(x)
            loss = loss_fn(y_logit, y)
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            profiler.step()
            y_pred = torch.softmax(y_logit, 1).argmax(1)
            train_acc += (y_pred == y).sum().item() / len (y_pred)
            #We can use another function if we want
    train_loss = train_loss / len(data)
    train_acc = train_acc / len(data)
    return train_loss, train_acc

In [16]:
def train(model: nn.Module, test_data: DataLoader, train_data: DataLoader, loss_fn:nn.Module,
        optimizer: torch.optim.Optimizer, device: torch.device, epochs: int
        , writer: SummaryWriter, log_dir: str) -> Dict[str, List]:
    """
    Trains and test a Pytorch model

    Args:
    -----
        model: A Pytorch model
        train_data: a DataLoader object with the train data
        test_data: a DataLoader object with the train data
        loss_fn: loss function to minimized
        optimizer: A Pytorch optimizer
        device: A target device to perform the operations ("cuda" or "cpu")
        epochs: A integre with the number of epochs that the model will be train
    Returns:
    --------
        A dictionary with the train and test loss and accuracy for every epoch
        in the form of 
        {"train_loss": [...],
        "train_acc": [...],
        "test_loss": [...],
        "test_acc": [...]}
    """
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_func(
            model,train_data,loss_fn,optimizer, device, log_dir)
        test_loss, test_acc = test_func(
            model, test_data, loss_fn, device)
        print(
            f"Epoch {epoch+1} |"
            f"train_loss :{train_loss: .4f} |"
            f"train_acc :{train_acc: .4f} |"
            f"test_loss :{test_loss: .4f} |"
            f"test_acc :{test_acc: .4f} "
        )
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ##Add of the writer
        if writer:
            writer.add_scalars(main_tag="Loss", 
            tag_scalar_dict={"train_loss": train_loss, "tests_loss": test_loss},
            global_step=epoch
            )

            writer.add_scalars(main_tag="Accuracy", 
            tag_scalar_dict={"train_acc": train_acc, "tests_acc": test_acc},
            global_step=epoch
            )

            writer.close()
    return results

Lets do some experiments with both the models and using different optimiztors, changing the epochs

In [9]:
OUT_FEATURES = len(class_names)

def create_effntb0():
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights).to(device)
    for param in model.features.parameters():
        param.requires_grad = False

    set_seed()

    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=0.2, inplace=True), #We dont change this variable
        torch.nn.Linear(in_features=1280, out_features=OUT_FEATURES, bias=True)
    ).to(device)
    print(f"[INFO] create new effntb0 model.")
    return model

def create_effntb2():
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)
    for param in model.features.parameters():
        param.requires_grad = False

    set_seed()

    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=0.3, inplace=True), #We dont change this variable
        torch.nn.Linear(in_features=1408, out_features=OUT_FEATURES, bias=True)
    ).to(device)
    print(f"[INFO] create new effntb2 model.")
    return model

def create_convnext_tiny():
    weights = torchvision.models.ConvNeXt_Tiny_Weights.DEFAULT
    model = torchvision.models.convnext_tiny(weights=weights).to(device)
    for param in model.features.parameters():
        param.requires_grad = False

    set_seed()

    model.classifier = torch.nn.Sequential(
        torch.nn.LayerNorm((768,), eps=1e-06, elementwise_affine=True),
        torch.nn.Flatten(start_dim=1, end_dim=-1),
        torch.nn.Linear(in_features=768, out_features=OUT_FEATURES, bias=True)
    ).to(device)
    print(f"[INFO] create new convnext_tiny model.")
    return model

def create_convnext_small():
    weights = torchvision.models.ConvNeXt_Small_Weights.DEFAULT
    model = torchvision.models.convnext_small(weights=weights).to(device)
    for param in model.features.parameters():
        param.requires_grad = False

    set_seed()

    model.classifier = torch.nn.Sequential(
        torch.nn.LayerNorm((768,), eps=1e-06, elementwise_affine=True),
        torch.nn.Flatten(start_dim=1, end_dim=-1),
        torch.nn.Linear(in_features=768, out_features=OUT_FEATURES, bias=True)
    ).to(device)
    print(f"[INFO] create new convnext_small model.")
    return model

In [10]:
def select_optimizer(model: nn.Module, optimizer: str):
    if optimizer == "Adam":
        return torch.optim.Adam(model.parameters(), lr=0.001)
    else: 
        return torch.optim.SGD(model.parameters(), lr=0.001)

In [11]:
effnetb0 = create_effntb0()
effnetb2 = create_effntb2()
convtiny = create_convnext_tiny()
convsmall = create_convnext_small()

[INFO] create new effntb0 model.
[INFO] create new effntb2 model.
[INFO] create new convnext_tiny model.
[INFO] create new convnext_small model.


In [12]:
torchinfo.summary(effnetb2, (32, 3, 224, 224), col_names=["num_params","trainable"]
                  ,row_settings=["var_names"])

Layer (type (var_name))                                      Param #                   Trainable
EfficientNet (EfficientNet)                                  --                        Partial
├─Sequential (features)                                      --                        False
│    └─Conv2dNormActivation (0)                              --                        False
│    │    └─Conv2d (0)                                       (864)                     False
│    │    └─BatchNorm2d (1)                                  (64)                      False
│    │    └─SiLU (2)                                         --                        --
│    └─Sequential (1)                                        --                        False
│    │    └─MBConv (0)                                       (1,448)                   False
│    │    └─MBConv (1)                                       (612)                     False
│    └─Sequential (2)                                        --    

Create the list with the information that we want to iterate

In [13]:
num_epochs = [2, 5, 10]
models_names = ["effnetb0", "effnetb2"]
optimizers = ["Adam", "SGD"]

We are going to use the same size and transformation for all the models for the sake of consistency

In [17]:
set_seed()

experiment_number = 0
model_num = 0
loss_fn = nn.CrossEntropyLoss().to(device)
timestamp = datetime.now().strftime("%Y-%m-%d")

for epochs in num_epochs:
    for model in models_names:
        for optimizer in optimizers:
            experiment_number += 1
            name = models_names[model_num]
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] model: {models_names[model_num]}")
            print(f"[INFO] Optimizer: {optimizer}")
            print(f"[INFO] Epochs: {epochs}")
            if model == "effnetb0":
                model = create_effntb0()
            else:
                model = create_effntb2()
            writer = create_write(name=optimizer, 
            model=models_names[model_num], extra=str(epochs))
            log_dir = os.path.join("log", timestamp + optimizer + name + str(epochs))
            optimizer = select_optimizer(model, optimizer)
            train(model, test_dataloader, train_dataloader, loss_fn, optimizer, device,
            epochs, writer, log_dir)
            print("-" * 50 + "\n")
    model_num = ~model_num

[INFO] Experiment number: 1
[INFO] model: effnetb0
[INFO] Optimizer: Adam
[INFO] Epochs: 2
[INFO] create new effntb0 model.
[INFO] create summary writer, saving to: runs/2024-08-23/Adam/effnetb0/2
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1 |train_loss : 1.7748 |train_acc : 0.2521 |test_loss : 1.3398 |test_acc : 0.3625 
Epoch 2 |train_loss : 1.0484 |train_acc : 0.3875 |test_loss : 1.0097 |test_acc : 0.4528 
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] model: effnetb0
[INFO] Optimizer: SGD
[INFO] Epochs: 2
[INFO] create new effntb2 model.
[INFO] create summary writer, saving to: runs/2024-08-23/SGD/effnetb0/2
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1 |train_loss : 1.1177 |train_acc : 0.2938 |test_loss : 1.0983 |test_acc : 0.3898 
Epoch 2 |train_loss : 1.1257 |train_acc : 0.3000 |test_loss : 1.0995 |test_acc : 0.3489 
--------------------------------------------------

[INFO] Experiment number: 3
[INFO] model: effnetb0
[INFO] Optimizer: Adam
[INFO] Epochs: 2
[INFO] create new effntb2 model.
[INFO] create summary writer, saving to: runs/2024-08-23/Adam/effnetb0/2
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


...

Epoch 1 |train_loss : 1.1177 |train_acc : 0.2938 |test_loss : 1.0983 |test_acc : 0.3898 
Epoch 2 |train_loss : 1.1257 |train_acc : 0.3000 |test_loss : 1.0995 |test_acc : 0.3489 
Epoch 3 |train_loss : 1.1109 |train_acc : 0.3250 |test_loss : 1.0966 |test_acc : 0.3483 
Epoch 4 |train_loss : 1.1160 |train_acc : 0.3375 |test_loss : 1.0948 |test_acc : 0.3483 
Epoch 5 |train_loss : 1.1066 |train_acc : 0.3521 |test_loss : 1.0937 |test_acc : 0.3659 
Epoch 6 |train_loss : 1.1101 |train_acc : 0.3500 |test_loss : 1.0914 |test_acc : 0.3875 
Epoch 7 |train_loss : 1.1101 |train_acc : 0.3667 |test_loss : 1.0914 |test_acc : 0.3688 
Epoch 8 |train_loss : 1.1038 |train_acc : 0.3771 |test_loss : 1.0886 |test_acc : 0.4051 
Epoch 9 |train_loss : 1.1181 |train_acc : 0.3458 |test_loss : 1.0868 |test_acc : 0.4023 
Epoch 10 |train_loss : 1.0967 |train_acc : 0.3646 |test_loss : 1.0864 |test_acc : 0.4051 
--------------------------------------------------

[INFO] Experiment number: 11
[INFO] model: effnetb0
[INFO

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 |train_loss : 1.7821 |train_acc : 0.2625 |test_loss : 1.2120 |test_acc : 0.3438 
Epoch 2 |train_loss : 1.0481 |train_acc : 0.4000 |test_loss : 1.0425 |test_acc : 0.4750 
Epoch 3 |train_loss : 1.2253 |train_acc : 0.3187 |test_loss : 1.0436 |test_acc : 0.4545 
Epoch 4 |train_loss : 1.2577 |train_acc : 0.2125 |test_loss : 1.0442 |test_acc : 0.4909 
Epoch 5 |train_loss : 1.1849 |train_acc : 0.2333 |test_loss : 1.0304 |test_acc : 0.5250 
Epoch 6 |train_loss : 1.1810 |train_acc : 0.2708 |test_loss : 1.0149 |test_acc : 0.5801 
Epoch 7 |train_loss : 1.1857 |train_acc : 0.2729 |test_loss : 1.0058 |test_acc : 0.5898 
Epoch 8 |train_loss : 1.1844 |train_acc : 0.2583 |test_loss : 0.9956 |test_acc : 0.6085 
Epoch 9 |train_loss : 1.1851 |train_acc : 0.2458 |test_loss : 0.9901 |test_acc : 0.5960 
Epoch 10 |train_loss : 1.1637 |train_acc : 0.2792 |test_loss : 0.9832 |test_acc : 0.6051 
--------------------------------------------------

[INFO] Experiment number: 12
[INFO] model: effnetb0
[INFO

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 |train_loss : 1.1177 |train_acc : 0.2938 |test_loss : 1.0983 |test_acc : 0.3898 
Epoch 2 |train_loss : 1.1257 |train_acc : 0.3000 |test_loss : 1.0995 |test_acc : 0.3489 
Epoch 3 |train_loss : 1.1109 |train_acc : 0.3250 |test_loss : 1.0966 |test_acc : 0.3483 
Epoch 4 |train_loss : 1.1160 |train_acc : 0.3375 |test_loss : 1.0948 |test_acc : 0.3483 
Epoch 5 |train_loss : 1.1066 |train_acc : 0.3521 |test_loss : 1.0937 |test_acc : 0.3659 
Epoch 6 |train_loss : 1.1101 |train_acc : 0.3500 |test_loss : 1.0914 |test_acc : 0.3875 
Epoch 7 |train_loss : 1.1101 |train_acc : 0.3667 |test_loss : 1.0914 |test_acc : 0.3688 
Epoch 8 |train_loss : 1.1038 |train_acc : 0.3771 |test_loss : 1.0886 |test_acc : 0.4051 
Epoch 9 |train_loss : 1.1181 |train_acc : 0.3458 |test_loss : 1.0868 |test_acc : 0.4023 
Epoch 10 |train_loss : 1.0967 |train_acc : 0.3646 |test_loss : 1.0864 |test_acc : 0.4051 
--------------------------------------------------

